In [1]:
import torch
import numpy
import sklearn

# When there is no missing values in the input time series

In [2]:
x = torch.randn(1, 1, 5)
print("Assume we input a time series, [batchSize= 1, column number = 1, length of the series = 5]")
x

Assume we input a time series, [batchSize= 1, column number = 1, length of the series = 5]


tensor([[[ 0.0553, -0.0302, -1.9991, -0.1097, -0.7812]]])

In [3]:
def chomp1D(x, chomp_size):
    return x[:, :, :-chomp_size]

Note: since weightnorm and relu activation function do not change shape, we omit them in this testing code for better clarity.

In [4]:
print("There are several Conv1 layers in the whole network. All of the conv1's out_channels=40 except the last conv1's out_channel=reduced_size=160.")

conv1 = torch.nn.Conv1d(in_channels=1, out_channels=160, kernel_size=3, padding=2, dilation=1)
conv1

There are several Conv1 layers in the whole network. All of the conv1's out_channels=40 except the last conv1's out_channel=reduced_size=160.


Conv1d(1, 160, kernel_size=(3,), stride=(1,), padding=(2,))

In [5]:
output_conv1 = conv1(x)
print(output_conv1.shape)
output_conv1

torch.Size([1, 160, 7])


tensor([[[ 0.5028,  0.4929,  0.1409,  ...,  1.3231,  0.4914,  0.8704],
         [ 0.0288,  0.0517, -0.2113,  ...,  0.5045, -0.3874,  0.2663],
         [-0.4488, -0.5000, -1.5966,  ..., -1.6309, -0.4846, -0.7582],
         ...,
         [ 0.1665,  0.1464,  0.3184,  ..., -0.3751,  0.4891, -0.0805],
         [-0.0182, -0.0467,  0.0271,  ...,  0.2835,  0.4184,  0.0718],
         [ 0.0733,  0.0767, -0.5916,  ..., -0.1320, -0.3647,  0.1018]]],
       grad_fn=<SqueezeBackward1>)

In [6]:
output_chomp1D = chomp1D(output_conv1, 2)
print(output_chomp1D.shape)
output_chomp1D

torch.Size([1, 160, 5])


tensor([[[ 5.0282e-01,  4.9286e-01,  1.4086e-01,  3.4896e-01,  1.3231e+00],
         [ 2.8781e-02,  5.1713e-02, -2.1128e-01, -1.1166e+00,  5.0452e-01],
         [-4.4884e-01, -4.9997e-01, -1.5966e+00, -4.6470e-01, -1.6309e+00],
         [ 1.0023e-01,  7.3756e-02, -6.2157e-01, -1.0542e-01, -5.7074e-01],
         [ 5.4731e-02,  7.5744e-02,  1.2276e+00,  1.0719e+00, -4.5056e-01],
         [-3.8510e-01, -3.6379e-01, -3.7000e-01, -1.1432e+00, -9.2701e-01],
         [-3.0202e-01, -3.4635e-01, -1.3438e+00, -3.5723e-01, -8.1518e-01],
         [ 5.7209e-02,  2.2417e-02, -4.1164e-01,  6.2231e-01,  1.0079e+00],
         [-3.2787e-01, -2.8782e-01,  1.6315e-01, -1.0393e+00, -1.1890e+00],
         [ 3.1724e-01,  3.1578e-01,  9.6562e-02,  1.4263e-03, -2.3204e-01],
         [ 1.7586e-01,  1.9580e-01,  6.0805e-01,  1.7057e-01,  8.4925e-01],
         [ 4.5029e-01,  4.1284e-01, -2.1712e-01,  7.5297e-01,  4.6609e-01],
         [ 3.0861e-01,  2.9685e-01, -6.7011e-01, -8.2267e-01, -6.5493e-01],
         [-5

In [7]:
reduce_size = torch.nn.AdaptiveMaxPool1d(1)

output_maxpool = reduce_size(output_chomp1D)

print(output_maxpool.shape)
print("As we can see, each input single value turn into 160 values, and these are the important features of this single value. The length of the input series is disregarded, ie 5 becomes 1. So now the input series is represented by 1 number that has 160 features.")
output_maxpool

torch.Size([1, 160, 1])
As we can see, each input single value turn into 160 values, and these are the important features of this single value. The length of the input series is disregarded, ie 5 becomes 1. So now the input series is represented by 1 number that has 160 features.


tensor([[[ 1.3231],
         [ 0.5045],
         [-0.4488],
         [ 0.1002],
         [ 1.2276],
         [-0.3638],
         [-0.3020],
         [ 1.0079],
         [ 0.1631],
         [ 0.3172],
         [ 0.8492],
         [ 0.7530],
         [ 0.3086],
         [ 0.5838],
         [ 0.5164],
         [ 1.0759],
         [ 1.1283],
         [ 0.5069],
         [ 0.5667],
         [ 1.5377],
         [-0.5124],
         [ 0.3474],
         [ 0.6573],
         [ 0.9657],
         [ 0.7789],
         [ 0.6769],
         [ 1.3309],
         [ 0.1231],
         [ 1.2434],
         [ 0.3378],
         [ 1.2515],
         [ 1.4095],
         [ 1.2538],
         [ 1.1985],
         [ 0.5482],
         [-0.1737],
         [ 0.3120],
         [ 1.1358],
         [ 0.4390],
         [ 0.1965],
         [ 1.0701],
         [ 0.7319],
         [ 0.0540],
         [ 1.0481],
         [ 0.0911],
         [ 0.3387],
         [ 0.3341],
         [-0.2482],
         [ 0.1200],
         [ 0.6040],


In [8]:
output_squeeze = output_maxpool.squeeze(2)
print(output_squeeze.shape)
print("For a single value of the input series, we don't need one bracket for each learnt feature, so we squeeze out the bracket. Now the features of a single input value becomes one vector.")
output_squeeze

torch.Size([1, 160])
For a single value of the input series, we don't need one bracket for each learnt feature, so we squeeze out the bracket. Now the features of a single input value becomes one vector.


tensor([[ 1.3231,  0.5045, -0.4488,  0.1002,  1.2276, -0.3638, -0.3020,  1.0079,
          0.1631,  0.3172,  0.8492,  0.7530,  0.3086,  0.5838,  0.5164,  1.0759,
          1.1283,  0.5069,  0.5667,  1.5377, -0.5124,  0.3474,  0.6573,  0.9657,
          0.7789,  0.6769,  1.3309,  0.1231,  1.2434,  0.3378,  1.2515,  1.4095,
          1.2538,  1.1985,  0.5482, -0.1737,  0.3120,  1.1358,  0.4390,  0.1965,
          1.0701,  0.7319,  0.0540,  1.0481,  0.0911,  0.3387,  0.3341, -0.2482,
          0.1200,  0.6040,  0.3575,  0.6982,  0.2079,  0.4032,  1.3808,  0.3828,
          0.0902,  1.4139,  0.1298,  0.8692,  0.3930,  1.2059,  0.1555, -0.2337,
          0.4658,  0.5107, -0.2310, -0.0237, -0.0630, -0.0879, -0.2717,  0.4232,
          0.5686,  1.3301, -0.1363,  1.1226,  0.3905,  0.8360,  0.6219,  0.6279,
          0.5661,  0.1570,  0.0958,  0.7620,  1.3436,  0.7149,  1.4459,  1.2893,
          0.4632,  0.2799, -0.1325,  1.3202,  1.2172,  0.8215,  0.0357,  1.2486,
          0.5523,  0.5563,  

In [9]:
# user-defined parameter which is the length they want their latent representation to be
reduced_size = 160
out_channels = 320

linear = torch.nn.Linear(in_features = reduced_size, out_features = out_channels)
output_linear = linear(output_squeeze)

print(output_linear.shape)
output_linear

torch.Size([1, 320])


tensor([[ 1.1423e+00, -1.8943e-01,  3.9639e-01,  3.3756e-01, -2.9349e-01,
         -4.1077e-01, -1.1545e+00,  1.6256e-01,  3.2954e-02,  3.2515e-01,
          1.9370e-01, -6.0564e-01,  3.9789e-02,  4.1061e-01,  4.6652e-01,
         -5.1951e-01,  4.5546e-02,  3.1762e-01, -1.7151e-01,  4.1700e-01,
         -3.3019e-01, -8.7405e-01, -6.5281e-01, -7.2817e-02,  7.8189e-01,
          2.9201e-01, -7.5600e-02,  2.6827e-01,  3.6822e-03,  3.3139e-01,
         -6.9273e-01,  5.6664e-01, -2.7964e-02, -4.6031e-02,  2.6289e-01,
         -3.8485e-01,  4.3438e-01, -9.6048e-02, -1.4830e-02, -2.8898e-01,
         -3.3543e-01, -6.8962e-02, -8.0126e-02, -6.9782e-01,  2.0653e-01,
          4.6897e-01,  4.4699e-01, -1.0856e-01, -5.0332e-01,  3.3947e-01,
         -5.9353e-01, -4.3386e-01,  3.8003e-01,  7.2433e-01,  3.3862e-01,
         -1.9343e-01, -3.1538e-01, -1.6528e-01, -1.6477e-01,  8.6288e-01,
         -5.8635e-02,  4.1432e-01, -5.0810e-01, -7.2204e-02,  3.4944e-01,
          1.9094e-01,  2.1328e-02,  3.

# When there is NaN value in the input series
AdaptiveMaxPool1d()will just return NaN for all inputs. <br>
This behavior is discussed and coded via:https://github.com/pytorch/pytorch/issues/7645

In [10]:
print("There are several Conv1 layers in the whole network. All of the conv1's out_channels=40 except the last conv1's out_channel=160.")
conv1 = torch.nn.Conv1d(in_channels=1, out_channels=160, kernel_size=3, padding=2, dilation=1)
conv1

There are several Conv1 layers in the whole network. All of the conv1's out_channels=40 except the last conv1's out_channel=160.


Conv1d(1, 160, kernel_size=(3,), stride=(1,), padding=(2,))

In [11]:
def chomp1D(x, chomp_size):
    return x[:, :, :-chomp_size]

In [12]:
x_withNaN = torch.randn(1, 1, 10)
print("Assume we input a time series, [batchSize= 1, column number = 1, length of the series = 10]")
x_withNaN

Assume we input a time series, [batchSize= 1, column number = 1, length of the series = 10]


tensor([[[-0.6887, -0.0834, -0.7951, -0.9995,  1.3388, -0.6010,  0.5827,
          -0.3108, -0.4429,  0.0976]]])

In [13]:
# set the 2nd value to be NaN
x_withNaN[0,0,1] = torch.log(torch.tensor([-1.]))
x_withNaN

tensor([[[-0.6887,     nan, -0.7951, -0.9995,  1.3388, -0.6010,  0.5827,
          -0.3108, -0.4429,  0.0976]]])

In [14]:
output_conv1 = conv1(x_withNaN)
print(output_conv1.shape)
output_conv1

torch.Size([1, 160, 12])


tensor([[[-0.7459,     nan,     nan,  ..., -0.7167, -0.4518, -0.5824],
         [-0.2732,     nan,     nan,  ...,  0.0880, -0.1842,  0.1432],
         [ 0.3341,     nan,     nan,  ...,  0.3934,  0.5066,  0.4486],
         ...,
         [ 0.5083,     nan,     nan,  ...,  0.3397,  0.4813,  0.5202],
         [-0.2204,     nan,     nan,  ..., -0.0588, -0.3568, -0.1415],
         [ 0.6168,     nan,     nan,  ...,  0.6892,  0.3781,  0.5815]]],
       grad_fn=<SqueezeBackward1>)

In [15]:
output_chomp1D = chomp1D(output_conv1, 2)
print(output_chomp1D.shape)
output_chomp1D

torch.Size([1, 160, 10])


tensor([[[-0.7459,     nan,     nan,  ..., -0.2658, -0.9259, -0.7167],
         [-0.2732,     nan,     nan,  ..., -0.5679,  0.2651,  0.0880],
         [ 0.3341,     nan,     nan,  ...,  0.5717,  0.2685,  0.3934],
         ...,
         [ 0.5083,     nan,     nan,  ...,  0.6035,  0.4901,  0.3397],
         [-0.2204,     nan,     nan,  ..., -0.6524,  0.1239, -0.0588],
         [ 0.6168,     nan,     nan,  ...,  0.1080,  0.9212,  0.6892]]],
       grad_fn=<SliceBackward>)

In [16]:
reduce_size = torch.nn.AdaptiveMaxPool1d(1)

output_maxpool = reduce_size(output_chomp1D)

print(output_maxpool.shape)
print("As we can see, each input single value turn into 160 values, and these are the important features of this single value. The length of the input series is disregarded, ie 10 becomes 1. So now the input series is represented by 1 number that has 160 features.")
output_maxpool

torch.Size([1, 160, 1])
As we can see, each input single value turn into 160 values, and these are the important features of this single value. The length of the input series is disregarded, ie 10 becomes 1. So now the input series is represented by 1 number that has 160 features.


tensor([[[nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan],
        

In [17]:
output_squeeze = output_maxpool.squeeze(2)
print(output_squeeze.shape)
print("For a single value of the input series, we don't need one bracket for each learnt feature, so we squeeze out the bracket. Now the features of a single input value becomes one vector.")
output_squeeze

torch.Size([1, 160])
For a single value of the input series, we don't need one bracket for each learnt feature, so we squeeze out the bracket. Now the features of a single input value becomes one vector.


tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<SqueezeBackward1>)

In [18]:
# user-defined parameter which is the length they want their latent representation to be
reduced_size = 160
out_channels=320

# Fracheschi messed up the order of in_features and out_features. I switched to correct one here.
linear = torch.nn.Linear(in_features = reduced_size, out_features = out_channels)
output_linear = linear(output_squeeze)

print(output_linear.shape)
output_linear

torch.Size([1, 320])


tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

# Unequal lengths vs Same length training set

## Same length example

In [19]:
x = torch.randn(3, 2, 4)
x[:, 0, 2:]=numpy.nan
print("Assume we input a time series, [batchSize= 3, column number = 2, length of the series = 4]")
print("As we can see, for each training set, the corresponding series all have same lengths. ie. all col1 has length 2, all col2 has length 4")
x

Assume we input a time series, [batchSize= 3, column number = 2, length of the series = 4]
As we can see, for each training set, the corresponding series all have same lengths. ie. all col1 has length 2, all col2 has length 4


tensor([[[ 0.8408, -0.6546,     nan,     nan],
         [-0.7015, -1.3145,  0.2572, -1.0638]],

        [[-1.1626,  0.5573,     nan,     nan],
         [ 2.6743,  1.3395, -0.2082,  0.8811]],

        [[-0.4517, -0.1752,     nan,     nan],
         [ 0.2501, -0.5502,  1.2579,  1.0505]]])

## Unequal lengths example:

In [20]:
x = torch.randn(3, 2, 4)
x[0, 0, 3]=numpy.nan
x[1, 1, 2:]=numpy.nan
print("Assume we input a time series, [batchSize= 3, column number = 2, length of the series = 4]")
print("The corresponding columns of training sets do not have the same length")
x

Assume we input a time series, [batchSize= 3, column number = 2, length of the series = 4]
The corresponding columns of training sets do not have the same length


tensor([[[-0.5961, -0.2652,  0.7300,     nan],
         [ 0.9152,  0.9476, -0.1323, -1.3774]],

        [[ 0.5036,  0.1680, -0.5816, -0.7644],
         [-0.5558,  0.3305,     nan,     nan]],

        [[ 0.6284, -0.0095,  0.4261,  0.2715],
         [-0.5495, -0.6978, -0.0104, -0.2112]]])

In [21]:
# scikit_wrappers.fit_encoder(X) will detect we have unequal lengths time series
varying = bool(numpy.isnan(numpy.sum(x.numpy())))
varying

True

Then `scikit_wrappers.fit_encoder(X`) will set:<br>
`loss = loss_varying( batch, self.encoder, train, save_memory=save_memory )`<br>
where `loss_varying = triplet_loss.TripletLossVaryingLength(compared_length, nb_random_samples, negative_penalty)`<br>
The need to distinguish between varying length from same length training sets is because we need to choose samples that are within series length (NaN not included), and when lengths are varying, we need to choose different lengths accordingly.

# Challenges 
1. Preserves only temporal order of values but not exact time of a value.
2. Cannot handle missing value in between the time series e.g. [2, 4, 5, NaN, 8, 0]. Only accepts time series where missing values are at begining or end.
3. Due to (1) and (2) above, it assumes equidistant time between values.

## Solution 1 - keep embedding method, change anomaly detection method
Use this embedding method and disregard exact time information, and a different anomaly detection method is needed.
#### Modify current anomaly detection algorithm:
Instead of using time interval to decide whether a value belongs or not, we can use a window of neighbor count.
#### Other anomaly algorithms:
- One-Class Support Vector Machine (Trained on normal data)
- Outlier detection with Local Outlier Factor (Computes local density of data)
- Kernel Density Estimation

## Solution 2 - change embedding method
1. Use call gaps to engineer a dataset only contains 'normal' sensor data.
2. Detect anomalies using method in paper "Learning Neural Representations for Network Anomaly Detection"

# Train the model with CPU (cuz I don't have Cuda)
Takes forever, and I am not exaggerating on the word "forever".

In [2]:
import os
import sys
# sys.path.append(os.getcwd())
# sys.path

In [6]:
# Check current working directory, make we are in the Fraceschi folder.
os.getcwd()

'C:\\Users\\Paws\\Desktop\\MasterProject\\Franceschi'

In [11]:
# If we aren't in Franceschi folder, use % cd to navigate to that folder.
%cd C:\\Users\\Paws\\Desktop\\MasterProject\\Franceschi

C:\Users\Paws\Desktop\MasterProject\Franceschi


In [ ]:
# Start training!
!python ucr.py --dataset Mallat --path C:\\Users\\Paws\\Desktop\\MasterProject\\ExtraDatasets\\UCRArchive_2018 --save_path C:\\Users\\Paws\\Desktop\\MasterProject\\Franceschi\\TrainedModel --hyper default_hyperparameters.json